In [5]:
import sqlite3
import pandas as pd
from IPython.display import display, Markdown

# --- CONFIGURACIÓN ---
# Ruta a tu base de datos
DB_PATH = 'data/database/parlamento.db'
# Número de filas a mostrar por tabla
LIMIT = 5

def display_database_overview(db_path, limit):
    """
    Se conecta a la BD y muestra las primeras N filas de cada tabla
    de manera ordenada en un notebook.
    """
    try:
        with sqlite3.connect(db_path) as conn:
            cursor = conn.cursor()
            
            # Obtener la lista de todas las tablas
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = [table[0] for table in cursor.fetchall()]
            
            display(Markdown(f"# 📊 Resumen de la Base de Datos: `{db_path}`"))
            display(Markdown(f"Mostrando las primeras **{limit} filas** de cada tabla."))
            
            for table in tables:
                if table.startswith('sqlite_'):
                    continue
                
                try:
                    # Usar pandas para leer la consulta SQL y obtener un DataFrame
                    df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT {limit}", conn)
                    
                    # Mostrar el nombre de la tabla como un título
                    display(Markdown(f"--- \n### 🏛️ Tabla: `{table}`"))
                    
                    if df.empty:
                        display(Markdown("_Esta tabla está actualmente vacía._"))
                    else:
                        # La función display() renderiza el DataFrame como una tabla HTML
                        display(df)
                        
                except Exception as e:
                    display(Markdown(f"No se pudo leer la tabla `{table}`. Error: {e}"))

    except sqlite3.Error as e:
        print(f"❌ Error de base de datos: {e}")

# --- Ejecutar la función para mostrar el resumen ---
display_database_overview(DB_PATH, LIMIT)

# 📊 Resumen de la Base de Datos: `data/database/parlamento.db`

Mostrando las primeras **5 filas** de cada tabla.

--- 
### 🏛️ Tabla: `dim_parlamentario`

,mp_uid,nombre_completo,nombre_propio,apellido_paterno,apellido_materno,genero,fecha_nacimiento,lugar_nacimiento,diputadoid,bcn_uri,url_foto,twitter_handle,sitio_web_personal,profesion,url_historia_politica,fecha_extraccion
0,1,María Candelaria Acevedo Sáez,María Candelaria,Acevedo,Sáez,Femenino,1958-09-12,Coronel,1096,http://datos.bcn.cl/recurso/persona/5247,https://www.bcn.cl/laborparlamentaria/imagen/1...,None,None,None,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-08-14
1,2,Eric Aedo Jeldres,Eric Mariano,Aedo,Jeldres,Masculino,1968-07-13,Los Ángeles,1097,http://datos.bcn.cl/recurso/persona/5253,https://www.bcn.cl/laborparlamentaria/imagen/1...,None,None,Administrador de Programas Sociales,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-08-14
2,3,Yovana Ahumada Palma,Yovana Alejandra,Ahumada,Palma,Femenino,1973-12-24,Antofagasta,1098,http://datos.bcn.cl/recurso/persona/5188,https://www.bcn.cl/laborparlamentaria/imagen/1...,yovyaftap2,None,Técnica agropecuaria,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-08-14
3,4,Jorge Alessandri Vergara,Jorge Iván,Alessandri,Vergara,Masculino,1979-06-08,Santiago de Chile,1009,http://datos.bcn.cl/recurso/persona/4604,https://www.bcn.cl/laborparlamentaria/imagen/1...,jalessandri,None,Abogado,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-08-14
4,5,René Alinco Bustos,René,Alinco,Bustos,Masculino,1958-06-02,,803,http://datos.bcn.cl/recurso/persona/135,https://www.bcn.cl/laborparlamentaria/imagen/1...,alinco_rene,None,Obrero de la Construcción,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-08-14


--- 
### 🏛️ Tabla: `parlamentario_aliases`

...

,voto_id,sesion_votacion_id,mp_uid,voto
0,1,32305,1,A Favor
1,2,32305,2,A Favor
2,3,32305,3,A Favor
3,4,32305,4,A Favor
4,5,32305,5,A Favor


--- 
### 🏛️ Tabla: `speech_turns`

_Esta tabla está actualmente vacía._

--- 
### 🏛️ Tabla: `interactions`

_Esta tabla está actualmente vacía._